## import and load

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import notebook
import datetime

#한글 깨짐방지
plt.rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [23]:
path = './data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission  = pd.read_csv(path + 'submission.csv')

## data setting

In [24]:
#time, installment, days_of_week, card_id, holyday는 필요없다 생각하여 글쓴이는 제거
train = train.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])
test = test.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])

In [25]:
# 'date','store_id'로 결합
train = train.groupby(['date','store_id']).agg({'amount':'sum'}).reset_index()
test = test.groupby(['date','store_id']).agg({'amount':'sum'}).reset_index()

In [26]:
#이러한 형태로 변환됨
train.head(1)

,date,store_id,amount
0,2016-08-01,2,2372


In [27]:
#date전환
train['date'] = pd.to_datetime(train['date'],infer_datetime_format= True)
test['date'] = pd.to_datetime(test['date'],infer_datetime_format= True)

In [28]:
# 나중을 위한 복제
train['temp_date'] = train['date']
test['temp_date'] = test['date']

In [29]:
# 날짜를 인덱스로 설정
train.set_index("date",inplace=True)
test.set_index("date",inplace=True)

### 일부제거
1. 거래기록이 160일 이하인 경우 삭제 -> 대회 목적이 100일 이후를 예측하는 것이므로 60일은 train(X), 100일은 predict(y)에 사용
    * 160일이 가장 이상적이라고 고려함
2. 2018-07-31 주변 5일 내 거래가 없으면 폐업으로 추정, 2018-07-31은 꼭 있어야하는 데이터

In [30]:
counter = 0
limit = 160

print("Before removing stores (due to limit): ", train.shape)

#160 이상이면 카운트를 올리고, 아니라면 제외함
for store_id in notebook.tqdm(range(train['store_id'].max()+1)):
    if train.loc[train['store_id'] == store_id, 'store_id' ].count() <= limit:
        counter += 1
    else :
        train = train[train.store_id !=store_id]

print(f"Total # of stores that exceeds {limit} is {counter}")
print("After removing stores (due to limit): ", train.shape)

Before removing stores (due to limit):  (480160, 3)



Total # of stores that exceeds 160 is 806
After removing stores (due to limit):  (47969, 3)


In [37]:
def keep_alive_store(df):
    store_id_list = df.store_id.unique()
    yes, no = 0, 0
    
    t2 = datetime.datetime.strptime('2018-07-31', '%Y-%m-%d')
    
    for store_id in store_id_list:
        if df.loc[df.store_id == store_id,'temp_date'].iloc[-1] == t2:
            yes += 1
        else:
            t1 = df.loc[df.store_id == store_id,'temp_date'].iloc[-1]
            difference = t2 - t1
            if difference.days <=5:
                yes += 1
            else :
                no += 1
                df = df[df.store_id != store_id]
    print(f"# of train store open/out of business: {yes}, {no}")
    return df

In [38]:
train = keep_alive_store(train)

# of train store open/out of business: 399, 382
